In [ ]:
import re
import gensim
import glob
import en_core_web_lg
import tqdm

In [ ]:
def decontracted(phrase):
    phrase = re.sub(r"won['’‘`]t", "will not", phrase)
    phrase = re.sub(r"can['’‘`]t", "can not", phrase)
    phrase = re.sub(r"ain['’‘`]t", "am not", phrase)

    phrase = re.sub(r"n['’‘`]t", " not", phrase)
    phrase = re.sub(r"['’‘`]re", " are", phrase)
    phrase = re.sub(r"['’‘`]s", " is", phrase)
    phrase = re.sub(r"['’‘`]d", " would", phrase)
    phrase = re.sub(r"['’‘`]ll", " will", phrase)
    phrase = re.sub(r"['’‘`]t", " not", phrase)
    phrase = re.sub(r"['’‘`]ve", " have", phrase)
    phrase = re.sub(r"['’‘`]m", " am", phrase)

    phrase = re.sub(r'[^\w.?!;]', ' ', phrase)
    phrase = re.sub(' +', ' ', phrase)
    sentences = re.split('([.;!?] *)', phrase)

    return ' '.join([i.capitalize() for i in  sentences])

In [ ]:
def prepare_english_text(input_file, output_file):
    nlp = en_core_web_lg.load(disable=['parser'])
    nlp.max_length = 5000000
    label_dict = {'NUM': 'ordinal1', 'PRON': 'pron1', 'PER': 'person1'}
    
    with open(input_file, 'r') as fin, open(output_file, 'w') as fout:
        for line in fin:
            
            preprocessed_text = decontracted(line.strip())
            nlp_doc = nlp(preprocessed_text)
            
            for token in nlp_doc:
                if token.pos_ in label_dict:
                    fout.write(label_dict[token.pos_])
                    fout.write(' ')
                    
                elif token.lemma_.isdigit():
                    fout.write('ordinal1')
                    fout.write(' ')
                    
                elif token.pos_ != 'PUNCT':
                    fout.write(token.lemma_.lower())
                    fout.write(' ')
                    
            fout.write('\n')

In [ ]:
def prepare_english_texts_from_folder(input_folder, output_folder):
    files = sorted(glob.glob(input_folder + '/*'))
    for file in tqdm.tqdm(files):
        output_file = output_folder + '/PREPROCESSED_' + file.split('/')[-1]
        prepare_russian_text(file, output_file)

In [ ]:
prepare_english_texts_from_folder('raw_english_texts', 'preprocessed_english_texts')